In [1]:
import cobra
import time
import numpy as np
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [2]:
import pickle 

# Reading model

In [3]:
cobra.Configuration().solver="cplex"

In [4]:
%%time
modelo=cobra.io.read_sbml_model("ACH_000714.xml")

CPU times: user 1min 19s, sys: 2.18 s, total: 1min 21s
Wall time: 1min 24s


In [5]:
model=modelo.copy()

In [1]:
#Remove compartment from metabolite ID (ONLY USE FOR ftINIT models)
'''
for i in range(len(model.metabolites)):
    m=model.metabolites[i]
    nombre=m.id
    m.id=nombre[:len(nombre)-3]
'''

'\nfor i in range(len(model.metabolites)):\n    m=model.metabolites[i]\n    nombre=m.id\n    m.id=nombre[:len(nombre)-3]\n'

In [7]:
#name of the original model. 
nombre="ACH_000714"

# Rephosphorylation of nucleoside triphosphates


Aerobic rephosphorylation of ATP from glucose

In [8]:
glucosa=model.reactions.MAR09034

In [9]:
O2=model.reactions.MAR09048

In [10]:
H2O=model.reactions.MAR09047

In [11]:
CO2=model.reactions.MAR09058

In [12]:
with model:   
    interes = Reaction('AerobicRephosphorylationOfATP')
    interes.lower_bound = 1.  #Se crea esta reacción de interés porque no estaba en el modelo
    interes.upper_bound = 1000.  
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1,
        model.metabolites.MAM02039c: 1,
        model.metabolites.MAM02751c: 1
    })
    model.add_reactions([interes])
    
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000

    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Aerobic-rephosphorylation-of-ATP-from-glucose.xml")
    print(sol)

0.0


Aerobic rephosphorylation of GTP

In [12]:
model.reactions.MAR04136

Reaction identifier,MAR04136
Name,GTP phosphohydrolase
Memory address,0x07fbb3e227b00
Stoichiometry,MAM02034e + MAM02040e --> MAM01948e + MAM02039e + MAM02751e GTP [Extracellular] + H2O [Extracellular] --> GDP [Extracellular] + H+ [Extracellular] + Pi [Extracellular]
GPR,ENSG00000054179 or ENSG00000079805 or ENSG00000087470 or ENSG00000101210 or ENSG00000106976 or...
Lower bound,0.0
Upper bound,1000.0


In [13]:
with model:
    interes = Reaction('AerobicRephosphorylationOfGTP')
    interes.lower_bound = 1.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM02034c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01948c: 1,
        model.metabolites.MAM02039c: 1,
        model.metabolites.MAM02751c: 1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Aerobic-rephosphorylation-of-GTP.xml")
    
    print(sol, interes)

0.0 AerobicRephosphorylationOfGTP: MAM02034c + MAM02040c --> MAM01948c + MAM02039c + MAM02751c


Aerobic rephosphorylation of CTP

In [14]:
interes=model.reactions.MAR04192

In [15]:
with model:
    interes = Reaction('CDPtoCTP')
    interes.name = 'CDPtoCTP '
    interes.lower_bound = 1.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01623c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01424c: 1,
        model.metabolites.MAM02039c: 1,
        model.metabolites.MAM02751c: 1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Aerobic-rephosphorylation-of-CTP.xml")
    print(sol)

0.0


Aerobic rephosphorylation of UTP

In [16]:
interes=model.reactions.MAR04061
interes

Reaction identifier,MAR04061
Name,UTP phosphohydrolase
Memory address,0x07fbb3e227b70
Stoichiometry,MAM02040e + MAM03130e --> MAM02039e + MAM02751e + MAM03106e H2O [Extracellular] + UTP [Extracellular] --> H+ [Extracellular] + Pi [Extracellular] + UDP [Extracellular]
GPR,ENSG00000138185 or ENSG00000168032 or ENSG00000188833
Lower bound,0.0
Upper bound,1000.0


Hay que volver a crear una reacción con los metabolitos en el citoplasma

In [17]:
with model:
    interes = Reaction('UDPtoUTP')
    interes.name = 'UDPtoUTP '
    interes.lower_bound = 1.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM03130c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM03106c: 1,
        model.metabolites.MAM02039c: 1,
        model.metabolites.MAM02751c: 1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Aerobic-rephosphorylation-of-UTP.xml")
    print(sol)

0.0


# De novo synthesis of nucleotides

ATP de novo synthesis

In [18]:
interes = model.reactions.MAR09038

In [19]:
NH3=model.reactions.MAR09073

In [20]:
Pi=model.reactions.MAR09072

Lo siguiente es un método alternativo a definir reacciones

In [21]:
with model:
    interes = Reaction('ATPdenovo')
    interes.name = 'ATPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    model.metabolites.MAM01371c.lower_bound=1 #ATP[c]
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-ATP-de-novo-synthesis.xml")
    print(sol)

0.0


CTP de novo synthesis

In [22]:
with model:
    interes = Reaction('CTPdenovo')
    interes.name = 'CTPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    
    CTPc = Reaction('CTPc')
    CTPc.lower_bound = 0  
    CTPc.upper_bound = 1000  
    CTPc.add_metabolites({
         model.metabolites.MAM01623c: -1.0 #GTP[c]
    })
    model.add_reactions([interes, CTPc])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    CTPc.lower_bound=1 #CTP[c]
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-CTP-de-novo-synthesis.xml")
    print(sol)

0.0


GTP de novo synthesis

In [23]:
with model:
    interes = Reaction('GTPdenovo')
    interes.name = 'GTPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    
    
    GTPc = Reaction('CTPc')
    GTPc.lower_bound = 0  
    GTPc.upper_bound = 1000  
    GTPc.add_metabolites({
         model.metabolites.MAM01623c: -1.0 #GTP[c]
    })
    model.add_reactions([interes, GTPc])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    GTPc.lower_bound=1 #GTP[c]
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-GTP-de-novo-synthesis.xml")
    print(sol)

0.0


UTP de novo synthesis

In [24]:
with model:
    interes = Reaction('UTPdenovo')
    interes.name = 'UTPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    
    
    UTPc = Reaction('UTPc')
    UTPc.lower_bound = 0  
    UTPc.upper_bound = 1000  
    UTPc.add_metabolites({
         model.metabolites.MAM03130c: -1.0 #UTP[c]
    })
    
    model.add_reactions([interes, UTPc])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    UTPc.lower_bound=1 #UTP[c]
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-UTP-de-novo-synthesis.xml")
    
    print(sol)

0.0


dATP de novo synthesis

In [25]:
with model:
    interes = Reaction('dATPdenovo')
    interes.name = 'dATPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    dATPc = Reaction('dATPc')
    dATPc.lower_bound = 0  
    dATPc.upper_bound = 1000  
    dATPc.add_metabolites({
         model.metabolites.MAM01642c: -1.0 #dATP[c]
    })
    model.add_reactions([interes, dATPc])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    dATPc.lower_bound=1 
    H2O.upper_bound=1000
    CO2.upper_bound=1000

    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-dATP-de-novo-synthesis.xml")
    print(sol)

0.0


dCTP de novo synthesis

In [26]:
with model:
    interes = Reaction('dCTPdenovo')
    interes.name = 'dCTPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })

    dCTPc = Reaction('dCTPc')
    dCTPc.lower_bound = 0  
    dCTPc.upper_bound = 1000  
    dCTPc.add_metabolites({
         model.metabolites.MAM01645c: -1.0 #dATP[c]
    })
    model.add_reactions([interes, dCTPc])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    dCTPc.lower_bound=1 
    H2O.upper_bound=1000
    CO2.upper_bound=1000

    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-dCTP-de-novo-synthesis.xml")
    print(sol)

0.0


dGTP de novo synthesis

In [27]:
with model:
    interes = Reaction('dGTPdenovo')
    interes.name = 'dGTPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    dGTPc = Reaction('dGTPc')
    dGTPc.lower_bound = 0  
    dGTPc.upper_bound = 1000  
    dGTPc.add_metabolites({
         model.metabolites.MAM01688c: -1.0 
    })
    model.add_reactions([interes, dGTPc])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    dGTPc.lower_bound=1 
    H2O.upper_bound=1000
    CO2.upper_bound=1000

    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-dGTP-de-novo-synthesis.xml")
    print(sol)

0.0


dTTP de novo synthesis

In [28]:
with model:
    interes = Reaction('dTTPdenovo')
    interes.name = 'dTTPdenovo'
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    dTTPc = Reaction('dTTPc')
    dTTPc.lower_bound = 0  
    dTTPc.upper_bound = 1000  
    dTTPc.add_metabolites({
         model.metabolites.MAM01753c: -1.0 
    })
    model.add_reactions([interes, dTTPc])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    Pi.lower_bound=-1000
    dTTPc.lower_bound=1 
    H2O.upper_bound=1000
    CO2.upper_bound=1000

    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-dTTP-de-novo-synthesis.xml")
    print(sol)

0.0


# Uptake of essential amino acids

Histidine uptake

In [29]:
histidina=model.reactions.MAR09038

In [30]:
with model:
    expulsar = Reaction('HistUptake')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02125c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    histidina.lower_bound=-1
    histidina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Histidine-uptake.xml")
    print(sol)

0.0


In [31]:
isoleucina=model.reactions.MAR09039

In [32]:
with model:
    expulsar = Reaction('IsoleucineUptake')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02184c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    isoleucina.lower_bound=-1
    isoleucina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Isoleucine-uptake.xml")
    print(sol)

0.0


In [33]:
leucina=model.reactions.MAR09040

In [34]:
with model:
    expulsar = Reaction('LeucineUptake')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02360c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    leucina.lower_bound=-1
    leucina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Leucine-uptake.xml")
    print(sol)

0.0


In [35]:
lisina=model.reactions.MAR09041

In [36]:
with model:
    expulsar = Reaction('LysineUptake')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02426c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    lisina.lower_bound=-1
    lisina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Lysine-uptake.xml")
    print(sol)

0.0


In [37]:
metionina=model.reactions.MAR09042

In [38]:
with model:
    expulsar = Reaction("MethionineUptake")
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02471c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    metionina.lower_bound=-1
    metionina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Methionine-uptake.xml")
    print(sol)

0.0


In [39]:
fenilalanina=model.reactions.MAR09043

In [40]:
with model:
    expulsar = Reaction("PhenylalanineUptake")
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02724c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    fenilalanina.lower_bound=-1
    fenilalanina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Phenyalanine-uptake.xml")
    print(sol)

0.0


In [41]:
treonina=model.reactions.MAR09044

In [42]:
with model:
    expulsar = Reaction("ThreonineUptake")
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02993c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    treonina.lower_bound=-1
    treonina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Threonine-uptake.xml")
    print(sol)

0.0


In [43]:
triptofano=model.reactions.MAR09045

In [44]:
with model:
    expulsar = Reaction("TryptophanUptake")
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM03089c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    triptofano.lower_bound=-1
    triptofano.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-TryptophanUptake.xml")
    print(sol)

0.0


In [45]:
valina=model.reactions.MAR09046

In [46]:
with model:
    expulsar = Reaction("ValineUptake")
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM03135c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    valina.lower_bound=-1
    valina.upper_bound=-1
    
    expulsar.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Valine-uptake.xml")
    print(sol)

0.0


# De novo synthesis of key intermediates

Glycerate 3-phosphate de novo syntesis

In [47]:
with model: 
    expulsar = Reaction('nueva')
    expulsar.lower_bound = 0.  
    expulsar.upper_bound = 1000. 
    expulsar.add_metabolites({
        model.metabolites.MAM00913c : -1.0,
    })
    model.add_reactions([expulsar])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    Pi.lower_bound=-1000
    
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    expulsar.lower_bound=1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-G3P-de-novo-synthesis.xml")
    print(sol)

0.0


### Mitochondrial acetyl-CoA de novo synthesis

In [48]:
acetilCoA=model.metabolites.MAM01261m
CoA=model.metabolites.MAM01597m

In [49]:
with model:
    interes = Reaction('mAcCoASynthesis')
    interes.name = 'mAcCoASynthesis'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        acetilCoA: -1,
        CoA: 1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound=1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Mitochondrial-AcCoA-synthesis.xml")
    print(sol, interes)

0.0 mAcCoASynthesis: MAM01261m --> MAM01597m


Mitochondrial AKG de novo synthesis

In [50]:
mAKG=model.metabolites.MAM01306m 

In [51]:
with model:
    interes = Reaction('mAKGSynthesis')
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        mAKG:-1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    interes.lower_bound=1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Mitochondrial-AKG-synthesis.xml")
    print(sol, interes)

0.0 mAKGSynthesis: MAM01306m --> 


Erythrose 4-phosphate de novo synthesis

In [52]:
eritrosa4P=model.metabolites.MAM01785c

In [53]:
with model:
    interes = Reaction('Eritrosa4Pdenovo')
    interes.name = 'Eritrosa4Pdenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        eritrosa4P: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Erythrose4P-de-novo.xml")
    print(sol, interes)

0.0 Eritrosa4Pdenovo: MAM01785c --> 


Fructose 6-phosphate de novo synthesis

In [54]:
fructosa6P=model.metabolites.MAM01845c

In [55]:
with model:
    interes = Reaction('Fructosa6Pdenovo')
    interes.name = 'Fructosa6Pdenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        fructosa6P: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Fructose6P-de-novo.xml")
    print(sol, interes)

0.0 Fructosa6Pdenovo: MAM01845c --> 


Glyceraldehyde 3-phosphate de novo synthesis

In [56]:
GAP=model.metabolites.MAM01939c

In [57]:
with model:
    interes = Reaction('GAPdenovo')
    interes.name = 'GAPdenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        GAP: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-GAP-de-novo.xml")
    print(sol, interes)

0.0 GAPdenovo: MAM01939c --> 


Glucose 6-phosphate de novo synthesis

In [58]:
glucosa6P=model.metabolites.MAM01968c

In [59]:
with model:
    interes = Reaction('Glucosa6Pdenovo')
    interes.name = 'Glucosa6Pdenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        glucosa6P: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Glucose6P-de-novo.xml")
    print(sol, interes)

0.0 Glucosa6Pdenovo: MAM01968c --> 


Mitochondrial oxaloacetate de novo synthesis

In [60]:
mOAA=model.metabolites.MAM02633m

In [61]:
with model:
    interes = Reaction('mOAAdenovo')
    interes.name = 'mOAAdenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
    mOAA: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Mitochondrial-OAA-de-novo.xml")
    print(sol, interes)

0.0 mOAAdenovo: MAM02633m --> 


Phosphoenolpyruvate de novo synthesis

In [62]:
PEP=model.metabolites.MAM02696c


In [63]:
with model:
    interes = Reaction('PEPdenovo')
    interes.name = 'PEPdenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
    PEP: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-PEP-de-novo.xml")
    print(sol, interes)

0.0 PEPdenovo: MAM02696c --> 


Pyruvate de novo synthesis

In [64]:
piruvato=model.metabolites.MAM02819c

In [65]:
with model:
    interes = Reaction('PyrDenovo')
    interes.name = 'PyrDenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
    piruvato: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Pyruvate-de-novo.xml")
    print(sol, interes)

0.0 PyrDenovo: MAM02819c --> 


Ribose 5-phosphate de novo synthesis

In [66]:
ribosa5P=model.metabolites.MAM02845c

In [67]:
with model:
    interes = Reaction('Rib5Penovo')
    interes.name = 'Rib5Pdenovo'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
    ribosa5P: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Ribose5P-de-novo.xml")
    print(sol, interes)

0.0 Rib5Penovo: MAM02845c --> 


Mitochondrial succinyl-CoA de novo synthesis

In [68]:
succinilCoA=model.metabolites.MAM02944m
mCoA=model.metabolites.MAM01597m

In [69]:
with model:
    interes = Reaction('mSuccCoASynthesis')
    interes.name = 'mSuccCoASynthesis'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        succinilCoA: -1.0,
        mCoA: 1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    Pi.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Mitochondrial-SuccCoA-synthesis.xml")
    print(sol)

0.0


# De novo synthesis of other compounds

Cholesterol de novo synthesis

In [70]:
colesterol=model.metabolites.MAM01450c
colesterol

Metabolite identifier,MAM01450c
Name,cholesterol [Cytosol]
Memory address,0x07fbb3e8f06a0
Formula,C27H46O
Compartment,c
In 13 reaction(s),"MAR00032, MAR10063, MAR00031, MAR01917, MAR01565, MAR01919, MAR01916, MAR08661, MAR01911, MAR01924, MAR01920, MAR00634, MAR01526"


In [71]:
with model:
    interes = Reaction('CholesterolSynthesis')
    interes.name = 'Cholesterol Synthesis'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        colesterol: -1
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    interes.lower_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Cholesterol-synthesis.xml")
    print(sol, interes)

0.0 CholesterolSynthesis: MAM01450c --> 


# Protein turnover

Protein synthesis from AAs

In [72]:
arginina=model.reactions.MAR09066
model.metabolites.MAM01365c

Metabolite identifier,MAM01365c
Name,arginine [Cytosol]
Memory address,0x07fbb3e8c2f28
Formula,C6H15N4O2
Compartment,c
In 46 reaction(s),"MAR05828, MAR05868, MAR05833, MAR11810, MAR06308, MAR04190, MAR05878, MAR05068, MAR02023, MAR06863, MAR06307, MAR09592, MAR05838, MAR05893, MAR05848, MAR05863, MAR06897, MAR05132, MAR07102,..."


In [73]:
tirosina=model.reactions.MAR09064
model.metabolites.MAM03101c

Metabolite identifier,MAM03101c
Name,tyrosine [Cytosol]
Memory address,0x07fbb3e6b70f0
Formula,C9H11NO3
Compartment,c
In 59 reaction(s),"MAR06768, MAR05130, MAR05542, MAR05524, MAR05701, MAR05533, MAR05700, MAR05478, MAR05699, MAR05593, MAR05698, MAR05804, MAR05697, MAR05696, MAR05668, MAR05704, MAR06770, MAR05744, MAR05703,..."


In [74]:
alanina=model.reactions.MAR09061
model.metabolites.MAM01307c

Metabolite identifier,MAM01307c
Name,alanine [Cytosol]
Memory address,0x07fbb3e935780
Formula,C3H7NO2
Compartment,c
In 74 reaction(s),"MAR05476, MAR05813, MAR06861, MAR05483, MAR05478, MAR05605, MAR05479, MAR05604, MAR05484, MAR05477, MAR05610, MAR05481, MAR05305, MAR05480, MAR05609, MAR05608, MAR05723, MAR05482, MAR05607,..."


In [75]:
glicina=model.reactions.MAR09067


In [76]:
serina=model.reactions.MAR09069


In [77]:
aspartato=model.reactions.MAR09097



In [78]:
glutamato=model.reactions.MAR09071



In [79]:
asparagina=model.reactions.MAR09062



In [80]:
glutamina=model.reactions.MAR09063



In [81]:
prolina=model.reactions.MAR09068



In [82]:
cisteina=model.reactions.MAR09065



In [83]:
aminoacidos=[arginina, histidina, lisina, metionina, fenilalanina, triptofano, 
             tirosina, alanina, glicina, serina, treonina, aspartato, glutamato, 
             asparagina, glutamina, isoleucina, leucina, prolina, valina ,cisteina]

In [84]:
urea=model.reactions.MAR09730



In [85]:
H2S=model.reactions.MAR09103


In [86]:
with model:
    albumina = Reaction('ProteinSynthesis')
    albumina.lower_bound = 0.  # This is the default
    albumina.upper_bound = 1000.  # This is the default
    albumina.add_metabolites({
        model.metabolites.MAM01308c: -1
        
    })
    model.add_reactions([albumina])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    for i in aminoacidos:
        i.lower_bound=-1000
    
    O2.lower_bound=-1000
    glucosa.lower_bound=-1000
    H2O.lower_bound=-1000
    NH3.lower_bound=-1000
    
    CO2.upper_bound=1000
    H2O.upper_bound=1000
    urea=upper_bound=1000
    H2S=upper_bound=1000
    
    albumina.lower_bound = 0.0001
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Protein-synthesis.xml")
    print(sol, albumina)

0.0 ProteinSynthesis: MAM01308c --> 


# Electron transport chain and TCA

Oxidative phosphorylation

In [87]:
succinato=model.metabolites.MAM02943m
mNADH=model.metabolites.MAM02553m
mNAD=model.metabolites.MAM02552m
fumarato=model.metabolites.MAM01862m
mH = model.metabolites.MAM02039m
mATP=model.metabolites.MAM01371m
mADP=model.metabolites.MAM01285m
mH2O=model.metabolites.MAM02040m
mPi=model.metabolites.MAM02751m

In [88]:
with model:
    interes = Reaction('OxidativePhosphorilation')
    interes.name = 'OxidativePhosphorilation'
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        mATP: -1.0,
        mH2O:-1.0,
        mADP: 1,
        mPi:1,
        mH:1
    })
    
    succinatoR = Reaction("Succinato")
    succinatoR.lower_bound=-1000
    succinatoR.upper_bound=1000
    succinatoR.add_metabolites({
        succinato: -1.0,
        mNADH:-1.0,
        mH: -1.0
        
    })
    
    fumaratoR = Reaction("Fumarato")
    fumaratoR.lower_bound=0
    fumaratoR.upper_bound=1000
    fumaratoR.add_metabolites({
        fumarato: -1.0,
        mNAD:-1.0
    })
    
    model.add_reactions([interes, succinatoR, fumaratoR])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    fumaratoR.lower_bound=1
    fumaratoR.upper_bound=1
    succinatoR.lower_bound=-1
    succinatoR.upper_bound=-1
    interes.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Oxidative-phosphorilation.xml")
    print(sol)

0.0


Oxidative decarboxylation

In [89]:
piruvato

Metabolite identifier,MAM02819c
Name,pyruvate [Cytosol]
Memory address,0x07fbb3e6d01d0
Formula,C3H3O3
Compartment,c
In 38 reaction(s),"MAR07141, MAR04388, MAR04198, MAR04210, MAR06627, MAR03974, MAR04358, MAR02027, MAR03883, MAR06016, MAR04171, MAR04930, MAR06015, MAR06020, MAR04089, MAR06019, MAR04360, MAR06018, MAR03851,..."


In [90]:
with model:
    entrada = Reaction('OxidativeDecarboxilation')
    entrada.lower_bound = -1000.  # This is the default
    entrada.upper_bound = 0.  # This is the default
    entrada.add_metabolites({
        model.metabolites.MAM02819m: -1.0,
        mNAD:-1.0,
        mCoA: -1
    })
    
    salida = Reaction("OxidativeDecarboxilation_2")
    salida.lower_bound=0
    salida.upper_bound=1000
    salida.add_metabolites({
        acetilCoA: -1.0,
        mNADH:-1.0
        
    })
    
    hidrogeno=Reaction("Hidrogeno")
    hidrogeno.lower_bound=0
    hidrogeno.upper_bpound=1000
    hidrogeno.add_metabolites({
        mH: -1.0
        
    })
    
    model.add_reactions([entrada, salida, hidrogeno])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    CO2.upper_bound=1
    CO2.lower_bound=1   
    
    entrada.lower_bound=-1
    entrada.upper_bound=-1
    
    salida.upper_bound=1
    salida.lower_bound=1
    
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Oxidative-decarboxilation.xml")
    print(sol)

0.0


Krebs cycle NADH

In [91]:
ubiquinone=model.metabolites.MAM03103m
ubiquinol=model.metabolites.MAM03102m
acetilCoA=model.metabolites.MAM01261m
mGTP=model.metabolites.MAM02034m
mGDP=model.metabolites.MAM01948m
mPi=model.metabolites.MAM02751m

In [92]:
with model:
    entrada = Reaction('TCA')
    entrada.lower_bound = -1000  # This is the default
    entrada.upper_bound = 1000.  # This is the default
    entrada.add_metabolites({
        acetilCoA: -1.0,
        mGDP:-1.0,
        ubiquinone: -1,
    })
    
    salida = Reaction("TCA_2")
    salida.lower_bound=-1000
    salida.upper_bound=1000
    salida.add_metabolites({
        ubiquinol: -1.0,
        mGTP:-1.0,
        mCoA: -1.0
        
    })
    
    NADH =Reaction("NADH")
    NADH.lower_bound=3
    NADH.upper_bound=3
    NADH.add_metabolites({
        mNADH:-1.0})
    
    
    NAD =Reaction("NAD")
    NAD.lower_bound=-3
    NAD.upper_bound=-3
    NAD.add_metabolites({
        mNAD:-1.0})
    
    Pi =Reaction("Pi")
    Pi.lower_bound=-1000
    Pi.upper_bound=1000
    Pi.add_metabolites({
        mPi:-1.0})
    
    hidrogeno=Reaction("Hidrogeno")
    hidrogeno.lower_bound=0
    hidrogeno.upper_bpound=1000
    hidrogeno.add_metabolites({
        mH: -1.0,
        model.metabolites.MAM02039c:-1     
    })
    model.add_reactions([entrada, salida, NADH, NAD, Pi, hidrogeno])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    H2O.lower_bound=-1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Krebs-cycle-NADH.xml")
    print(sol)

0.0


Ubiquinol-to-proton

In [93]:
ubiquinone

Metabolite identifier,MAM03103m
Name,ubiquinone [Mitochondria]
Memory address,0x07fbb3e6b7358
Formula,C59H90O4
Compartment,m
In 21 reaction(s),"MAR02366, MAR02367, MAR03783, MAR00563, MAR00483, MAR04575, MAR03211, MAR04652, MAR03838, MAR03769, MAR02373, MAR02035, MAR02372, MAR06921, MAR04242, MAR02369, MAR06918, MAR02370, MAR03751,..."


In [94]:
with model:
    entrada = Reaction('Ubiquinol')
    entrada.lower_bound = -1000 # This is the default
    entrada.upper_bound = 0  # This is the default
    entrada.add_metabolites({
        ubiquinol: -1.0       
    })
    
    salida = Reaction("Ubiquinona")
    salida.lower_bound=0
    salida.upper_bound=1000
    salida.add_metabolites({
        ubiquinone: -1.0       
    })
        
    hidrogeno=Reaction("Hidrogeno_mit")
    hidrogeno.lower_bound=-1000
    hidrogeno.upper_bound=1000
    hidrogeno.add_metabolites({
        mH: -1.0
        
    })
    
    hidrogeno_c=Reaction("Hidrogeno_cit")
    hidrogeno_c.lower_bound=6
    hidrogeno_c.upper_bound=1000
    hidrogeno_c.add_metabolites({
        model.metabolites.MAM02039c:-1 
        
    })
    model.add_reactions([entrada, salida, hidrogeno, hidrogeno_c])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    entrada.lower_bound =-1
    entrada.upper_bound =-1
    salida.lower_bound = 1
    salida.upper_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Ubiquinol-to-proton.xml")
    print(sol)

0.0


Ubiquinol-to-ATP

In [95]:
with model:
    entrada = Reaction('Ubiquinol')
    entrada.lower_bound = -1000 # This is the default
    entrada.upper_bound = 1000.  # This is the default
    entrada.add_metabolites({
        ubiquinol: -1.0       
    })
    
    salida = Reaction("Ubiquinona")
    salida.lower_bound=0
    salida.upper_bound=1000
    salida.add_metabolites({
        ubiquinone: -1.0       
    })
        
    hidrogeno=Reaction("Hidrogeno")
    hidrogeno.lower_bound=0
    hidrogeno.upper_bpound=1000
    hidrogeno.add_metabolites({
        mH: -1.0,
        model.metabolites.MAM02039c:-1     
    })
    
    model.add_reactions([entrada, salida, hidrogeno])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    entrada.lower_bound =-1
    entrada.upper_bound =-1
    salida.lower_bound = 1
    salida.upper_bound = 1
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Ubiquinol-to-ATP.xml")
    print(sol)

0.0


# Beta oxidation of fatty acids

Beta oxidation of saturated FA

In [13]:
stereate=model.reactions.MAR00248

In [14]:
with model:
    interes = Reaction('BOxidationSatFA')
    interes.lower_bound = 0  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    model.add_reactions([interes])
    
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    O2.lower_bound=-1000
    stereate.lower_bound=-1
    stereate.upper_bound=-1
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-BOxidationSatFA.xml")
    print(sol, interes)

0.0 BOxidationSatFA: MAM01371c + MAM02040c --> MAM01285c + MAM02039c + MAM02751c


Beta oxidation of long-chain FA

In [98]:
tetracoAcid =  model.reactions.MAR13050
model.metabolites.MAM00315e


Metabolite identifier,MAM00315e
Name,"12,15,18,21-tetracosatetraenoic acid..."
Memory address,0x07fbb3e989f28
Formula,C24H39O2
Compartment,e
In 4 reaction(s),"MAR00015, MAR13050, MAR00395, MAR00392"


In [99]:
with model:
    for r in model.exchanges:
        r.knock_out()
    tetracoAcid.lower_bound=-1
    tetracoAcid.upper_bound=-1
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Beta-oxidation-of-long-chain-FA.xml")
    print(sol)

0.0


Beta oxidation of odd-chain FA

In [100]:
margaricAcid= model.reactions.MAR00620
model.metabolites.MAM02456e

Metabolite identifier,MAM02456e
Name,margaric acid [Extracellular]
Memory address,0x07fbb3e7bb2e8
Formula,C17H33O2
Compartment,e
In 4 reaction(s),"MAR00239, MAR00015, MAR00620, MAR00236"


In [101]:
with model:
    interes = Reaction('BOxidationODFA')
    interes.lower_bound = 0  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.knock_out()
    margaricAcid.lower_bound=-1
    margaricAcid.upper_bound=-1
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-BOxidationOddFA.xml")
    print(sol)

0.0


Beta oxidation of unsaturated fatty acid (n-9) 

In [102]:
ximenicAcid=model.reactions.MAR13063
model.metabolites.MAM03153e

Metabolite identifier,MAM03153e
Name,ximenic acid [Extracellular]
Memory address,0x07fbb3e647a90
Formula,C26H49O2
Compartment,e
In 4 reaction(s),"MAR00015, MAR00347, MAR13063, MAR00344"


In [103]:
with model:
    for r in model.exchanges:
        r.knock_out()
    ximenicAcid.lower_bound=-1
    ximenicAcid.upper_bound=-1
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-BOxidationUnSatFAn-9.xml")
    print(sol)

0.0


Beta oxidation of unsaturated fatty acid (n-6) 

In [104]:
linoleate=model.reactions.MAR09035
model.metabolites.MAM02387e

Metabolite identifier,MAM02387e
Name,linoleate [Extracellular]
Memory address,0x07fbb3e79b3c8
Formula,C18H31O2
Compartment,e
In 5 reaction(s),"MAR00399, MAR00015, MAR07152, MAR09035, MAR00396"


In [105]:
with model:
    interes = Reaction('BOxidationInSatFA')
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.knock_out()
    linoleate.lower_bound=-1
    linoleate.upper_bound=-1
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-BOxidationUnSatFAn-6.xml")
    print(sol, interes)

0.0 BOxidationInSatFA: MAM01371c + MAM02040c --> MAM01285c + MAM02039c + MAM02751c


Uptake and beta oxidation of all NEFAs

In [15]:
NEFA_In = model.reactions.MAR09033


In [16]:
NEFA_In2=model.reactions.MAR09056
model.metabolites.MAM02034c

Metabolite identifier,MAM02034c
Name,GTP [Cytosol]
Memory address,0x07fe344b168d0
Formula,C10H12N5O14P3
Compartment,c
In 17 reaction(s),"MAR04042, MAR10064, MAR04022, MAR04160, MAR03967, MAR04387, MAR05301, MAR04171, MAR07867, MAR09801, MAR04614, MAR04168, MAR04169, MAR04401, MAR07792, MAR04016, MAR04182"


In [17]:
with model:
    interes = Reaction('BOxidationNEFAs')
    interes.lower_bound = 0.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    model.add_reactions([interes])
    for r in model.exchanges:
        r.knock_out()
    NEFA_In.lower_bound=-1
    #NEFA_In.upper_bound=-1
    NEFA_In2.lower_bound=-1
    #NEFA_In2.lower_bound=-1  
    O2.lower_bound=-1000
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-BOxidationNEFAs.xml")
    print(sol, interes)

0.0 BOxidationNEFAs: MAM01371c + MAM02040c --> MAM01285c + MAM02039c + MAM02751c


# De novo synthesis of phospholipids

Choline uptake

In [109]:
model.metabolites.MAM01513e
colina=model.reactions.MAR09083

In [110]:
with model:
    expulsar = Reaction('CholineUptake')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM01513c: -1.0,
    })
    model.add_reactions([expulsar])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    colina.lower_bound=-1
    colina.upper_bound=-1
    
    expulsar.lower_bound=1
    expulsar.upper_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-CholineUptake.xml")
    print(sol)

0.0


Inositol uptake

In [35]:
inositol=model.reactions.MAR09361
model.metabolites.MAM02171e

Metabolite identifier,MAM02171e
Name,inositol [Extracellular]
Memory address,0x07f8b3898acf8
Formula,C6H12O6
Compartment,e
In 5 reaction(s),"MAR09611, MAR08912, MAR09361, MAR08911, MAR04861"


In [112]:
with model:
    expulsar = Reaction('InositolUptake')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02171c: -1.0,
    })
    model.add_reactions([expulsar])
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
    
    inositol.lower_bound=-1
    inositol.upper_bound=-1
    
    expulsar.lower_bound=1
    expulsar.upper_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-InositolUptake.xml")
    print(sol)

0.0


Phosphatidylcholine de novo synthesis

In [113]:
PC_LD=model.reactions.MAR00655

In [33]:
Pi=model.reactions.MAR09072

In [115]:
with model:
    interes = Reaction('PCdenovo')
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    PC = Reaction("PC")
    PC.lower_bound = 0 # This is the default
    PC.upper_bound = 1000.  # This is the default
    PC.add_metabolites({
       model.metabolites.MAM02684c: -1.0
    })
    model.add_reactions([interes, PC])
    
    
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    NEFA_In2.lower_bound=-1000
    NEFA_In.lower_bound=-1000
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    colina.lower_bound=-1000
    
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    PC.lower_bound=1

    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Phosphatidylcholine-de-novo-synthesis.xml")
    print(sol, interes, PC)

0.0 PCdenovo: MAM01371c + MAM02040c <=> MAM01285c + MAM02039c + MAM02751c PC: MAM02684c --> 


Phosphatidylethanolamine de novo synthesis

In [116]:
etanolamina=model.reactions.MAR09084
model.metabolites.MAM01797e

Metabolite identifier,MAM01797e
Name,ethanolamine [Extracellular]
Memory address,0x07fbb3eb185c0
Formula,C2H8NO
Compartment,e
In 2 reaction(s),"MAR09084, MAR07896"


In [117]:
PE_LD=model.metabolites.MAM02685c

In [118]:
with model:
    interes = Reaction('PEdenovo')
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    
    PE = Reaction("PE")
    PE.lower_bound = -1000 # This is the default
    PE.upper_bound = 1000.  # This is the default
    PE.add_metabolites({
        PE_LD:-1.0
        
    })
    model.add_reactions([interes, PE])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    NEFA_In.lower_bound=-1000 
    NEFA_In2.lower_bound=-1000
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    etanolamina.lower_bound=-1000
    
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    PE.lower_bound=1
    PE.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Phosphatidylethanolamine-de-novo-synthesis.xml")
    print(sol, interes, PE)

0.0 PEdenovo: MAM01371c + MAM02040c <=> MAM01285c + MAM02039c + MAM02751c PE: MAM02685c --> 


Phosphatidylserine de novo synthesis

In [119]:
PS_LD=model.metabolites.MAM02808e

In [120]:
with model:
    interes = Reaction('PSdenovo')
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    
    PS = Reaction("PS")
    PS.lower_bound = -1000 # This is the default
    PS.upper_bound = 1000.  # This is the default
    PS.add_metabolites({
        PS_LD:-1.0
        
    })
    model.add_reactions([interes, PS])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    NEFA_In.lower_bound=-1000 
    NEFA_In2.lower_bound=-1000
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    serina.lower_bound=-1000
    
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    PS.lower_bound=1
    PS.upper_bound=1000
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Phosphatidylserine-de-novo-synthesis.xml")
    print(sol, interes, PS)

0.0 PSdenovo: MAM01371c + MAM02040c <=> MAM01285c + MAM02039c + MAM02751c PS: MAM02808e --> 


Phosphatidylinositol de novo synthesis

In [31]:
PI_pool=model.metabolites.MAM02750e
PI_pool

Metabolite identifier,MAM02750e
Name,PI pool [Extracellular]
Memory address,0x07f8b38775da0
Formula,C11H16O13PR2
Compartment,e
In 1 reaction(s),MAR12127


In [36]:
with model:
    interes = Reaction('PIdenovo')
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })

    PI = Reaction("PI")
    PI.bounds=[1,1000]
    PI.add_metabolites({
        PI_pool:-1.0
    })
    model.add_reactions([interes, PI])
    for r in model.exchanges:
        r.bounds=[0,0]

    NEFA_In.lower_bound=-1000
    NEFA_In2.lower_bound=-1000
    glucosa.lower_bound=-1000
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    inositol.lower_bound=-1000

    H2O.upper_bound=1000
    CO2.upper_bound=1000


    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Phosphatidylinositol-de-novo-synthesis.xml")
    print(sol, interes, PI)

0.0 PIdenovo: MAM01371c + MAM02040c <=> MAM01285c + MAM02039c + MAM02751c PI: MAM02750e --> 


In [37]:
modelo=cobra.io.read_sbml_model("ACH_000714-Phosphatidylinositol-de-novo-synthesis.xml")
dr=single_gene_deletion(modelo)
nombre="MT52-SGD-mieloma"
with open(nombre, "wb") as f:
    pickle.dump(dr, f)

# Vitamins and co-factors

Thiamin phosphorylation to TPP

In [123]:
tiamina=model.reactions.MAR09159

In [124]:
with model:
    interes = Reaction('ThiaminToTPP')
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    
    expulsar = Reaction('ThiaminPP')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02984c: -1.0,
    })
    
    hidrogeno=Reaction("Hidrogeno")
    hidrogeno.lower_bound=-1000
    hidrogeno.upper_bound=1000
    hidrogeno.add_metabolites({
        model.metabolites.MAM02039c:1     
    })
    
    model.add_reactions([interes, hidrogeno, expulsar])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    tiamina.lower_bound=-1000  
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    expulsar.lower_bound=1
    H2O.lower_bound=-1000
    H2O.upper_bound=1000
    
    model.reactions.MAR09079.lower_bound=-1000
    model.reactions.MAR09079.upper_bound=1000
    
    hidrogeno.lower_bound=-1000
    hidrogeno.upper_bound=1000
    
    sol=model.optimize()
    cobra.io.write_sbml_model(model,nombre+"-Thiamin-phosphorylation-to-TPP.xml")
    print(sol, interes)

<Solution 0.000 at 0x7fbb40265710> ThiaminToTPP: MAM01371c + MAM02040c <=> MAM01285c + MAM02039c + MAM02751c


Coenzyme A synthesis from pantothenate

In [125]:
pantotenato=model.reactions.MAR09145

In [126]:
with model:
    interes = Reaction('ThiaminToTPP')
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM01371c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01285c: 1.0,
        model.metabolites.MAM02751c: 1.0,
        model.metabolites.MAM02039c: 1.0
    })
    
    expulsar = Reaction('CoA')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM01597c: -1.0,
    })

    model.add_reactions([interes, expulsar])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    pantotenato.lower_bound=-1000  
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    NH3.lower_bound=-1000
    cisteina.lower_bound=-1000
    H2O.lower_bound=-1000
    glucosa.lower_bound=-1000
    
    expulsar.lower_bound=1
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Coenzyme-A-synthesis-from-pantothenate.xml")
    print(sol, interes)

0.0 ThiaminToTPP: MAM01371c + MAM02040c <=> MAM01285c + MAM02039c + MAM02751c


FAD synthesis from riboflavin

In [127]:
riboflavina=model.reactions.MAR09143

In [128]:
with model:
    expulsar = Reaction('FAD')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM01802c: -1.0,
    })

    model.add_reactions([expulsar])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    riboflavina.lower_bound=-1000  
    O2.lower_bound=-1000
    Pi.lower_bound=-1000
    NH3.lower_bound=-1000
    glucosa.lower_bound=-1000
    expulsar.lower_bound=1
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-FAD-synthesis-from-riboflavin.xml")
    print(sol, interes)

0.0 ThiaminToTPP: MAM01371c + MAM02040c <=> MAM01285c + MAM02039c + MAM02751c


Heme biosynthesis

In [129]:
fe2=model.reactions.MAR09076

In [130]:
with model:
    interes = Reaction('HemeBiosynthesis')
    interes.lower_bound = -1000.  # This is the default
    interes.upper_bound = 1000.  # This is the default
    interes.add_metabolites({
        model.metabolites.MAM02034c: -1.0,
        model.metabolites.MAM02040c: -1.0,
        model.metabolites.MAM01948c: 1,
        model.metabolites.MAM02039c: 1,
        model.metabolites.MAM02751c: 1
        
    })
    
    expulsar = Reaction('Heme')
    expulsar.lower_bound = 0.  # This is the default
    expulsar.upper_bound = 1000.  # This is the default
    expulsar.add_metabolites({
        model.metabolites.MAM02049c: -1.0,
    })

    model.add_reactions([interes, expulsar])
    for r in model.exchanges:
        r.lower_bound=0
        r.upper_bound=0
        
    fe2.lower_bound=-1000  
    O2.lower_bound=-1000
    NH3.lower_bound=-1000
    glucosa.lower_bound=-1000
    
    expulsar.lower_bound=1
    H2O.upper_bound=1000
    CO2.upper_bound=1000
    
    
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-Heme-biosynthesis.xml")
    print(sol, interes)

0.0 HemeBiosynthesis: MAM02034c + MAM02040c <=> MAM01948c + MAM02039c + MAM02751c


# Biomass Production under Ham's medium

In [ ]:
mediaComps ={'glucose',
             'arginine',
             'histidine',
             'lysine',
             'methionine',
             'phenylalanine',
             'tryptophan',
             'tyrosine',
             'alanine',
             'glycine',
             'serine',
             'threonine',
             'aspartate',
             'glutamate',
             'asparagine',
             'glutamine',
             'isoleucine',
             'leucine',
             'proline',
             'valine',
             'cysteine',
             'thiamin',
             'hypoxanthine',
             'folate',
             'biotin',
             'pantothenate',
             'choline',
             'inositol',
             'nicotinamide',
             'pyridoxine',
             'riboflavin',
             'thymidine',
             'aquacob(III)alamin',
             'lipoic acid',
             'sulfate',
             'linoleate',
             'linolenate',
             'O2',
             'H2O',
             'retinoate',
             'Fe2+',
             'Pi',
             'alpha-tocopherol',
             'gamma-tocopherol'}

In [ ]:
RComps=[]
for i,r in enumerate(model.reactions):
    if len(r.metabolites)==1:
        m=list(r.metabolites)[0]
        if m.id in mediaComps or m.name in mediaComps:
            RComps.append(r)

In [ ]:
#Use for Human1
with model:
    RComps=[]
    for i,r in enumerate(model.reactions):
        if len(r.metabolites)==1:
            m=list(r.metabolites)[0]
            if m.id in mediaComps or m.name in mediaComps:
                RComps.append(r)

    for r in set(model.exchanges).difference(RComps):
        r.lower_bound=0
    biomass=model.reactions.MAR13082
    biomass.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-BiomassProduction.xml")
    print(sol, interes)

In [ ]:
#Use for models from ftINIT
with model:
    nuevo_medio=set()
    for i in mediaComps:
        i=i+" [Extracellular]"
        new_medium.add(i)

    RComps=[]
    for i,r in enumerate(model.reactions):
        if len(r.metabolites)==1:
            m=list(r.metabolites)[0]
            if m.id in new_medium or m.name in new_medium:
                RComps.append(r)

    for r in set(model.exchanges).difference(RComps):
        r.lower_bound=0
    biomass=model.reactions.MAR13082
    biomass.lower_bound=1
    sol=model.slim_optimize()
    cobra.io.write_sbml_model(model,nombre+"-BiomassProduction.xml")
    print(sol, interes)